In [1]:
import os
import json
import psycopg2 as pg2
from time import sleep
from crawler import Crawler
from bs4 import BeautifulSoup

brands = [
    'Gibson',
    'Fender',
    'Rickenbacker',
    'Ibanez',
    'Epiphone',
    'Gretsch',
    'Paul Reed Smith',
    'Reverend',
    'Schecter',
    'Guild'
]

# Load login credentials
with open('config.json', 'r') as f:
    conf = json.load(f)

In [4]:
conn = pg2.connect(dbname='reverb', host='192.168.0.209', password='galvanize', user='postgres')
cur = conn.cursor()
conn.autocommit = True
cur.execute("""CREATE TABLE guitars(
               id INT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
               
           );""")
conn.close()

In [2]:
def hot_soup():
    """Return a beautiful soup object from the driver's current source"""
    return BeautifulSoup(driver.page_source, 'html.parser')

def login():
    """Login to Reverb.com"""
    soup = hot_soup()
    
    # Find login button
    start = soup.find('a', class_='site-header__nav__link--login')
    driver.bsel(start).click()
    sleep(1)
    soup = hot_soup()
    
    # Find Username & Password fields
    usn = soup.find(id='user_session_login')
    psw = soup.find(id='user_session_password')
    username = driver.bsel(usn)
    password = driver.bsel(psw)
    
    # Enter in credentials & submit
    username.send_keys(conf['REVERB_LOGIN'])
    password.send_keys(conf['REVERB_PASS'])
    submit = soup.find('input', attrs={'type': 'submit'})
    driver.bsel(submit).click()

def scrape_links(soup):
    tiles = soup.find('ul', class_='tiles')
    links = tiles.find_all('a', class_='csp-square-card__inner')
    
    return {l.find('h3').text: l['href'] for l in links}

def scrape_transactions(soup):
    transactions = soup.find_all('tr', class_='transaction')
    output = []
    if transactions:
        for transaction in transactions:
            date = transaction.find(class_='date').text
            cond = transaction.find(class_='condition').text
            price = transaction.find(class_='final').text
            output.append((date, cond, price))
    
    return output

In [ ]:
driver = Crawler()
driver.get('https://reverb.com/price-guide/electric-guitars')
links = {}
try:
    login()
    sleep(1)
    driver.get('https://reverb.com/price-guide/guide/685-fender-jazzmaster-1960-sunburst')
    sleep(1)
    soup = hot_soup()
    
except Exception as e:
    print(e)

# Collect links for all price guide listings
# while True:
#     links.update(scrape_links(hot_soup()))
#     nxt = soup.find('li', class_='pagination__page pagination__page--next')
#     if nxt:
#         # Click next button
#         driver.bsel(nxt).click()
#         sleep(2)
#     else:
#         # All links have been gathered
#         break

for title, link in links.items():
    
    # Collect all transactions from a price guide
    while True:
        soup = hot_soup()
        transactions = scrape_transactions(soup)
        print(transactions)
        nxt = soup.find('button', attrs={'title': 'Next'})
        if 'disabled' in nxt.attrs:
            # All transactions have been collected
            break
        else:
            # Click next button
            driver.bsel(nxt).click()

driver.cleanup()

In [ ]:
'disabled' in (soup.find('button', attrs={'title': 'Next'}).attrs)

In [ ]:
while True:
    transactions = scrape_transactions(soup)
    print(transactions)
    nxt = soup.find('button', attrs={'title': 'Next'})
    if 'disabled' in nxt.attrs:
        # All transactions have been collected
        break
    else:
        # Click next button
        driver.bsel(nxt).click()